In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import utils

import data_mappers
import nn_models
tf.logging.set_verbosity(tf.logging.INFO)
tf.__version__

In [ ]:
import importlib
importlib.reload(data_mappers)
importlib.reload(nn_models)
importlib.reload(utils)

from nn_models import build_mlp_model_fn
from data_mappers import Window_Creator

In [ ]:
# Load recordings
recs = sorted(utils.enumerate_recording_paths('../dataset/raw/*record*'))
recs += sorted(utils.enumerate_recording_paths('../dataset/raw/benni*/*record*'))
# Create feeder
print(f'Number of recordings: {len(recs)}')
feeder = Window_Creator(rec_paths=recs, window_size=120, subsample=1)


In [ ]:
# Create model
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=10000)

# Specify number of layers
dense_layers = (20, 20)
# Specify learning rate
learning_rate=0.01
# Specify the decay rate
rho=0.95

layer_conf = '-'.join((str(u) for u in dense_layers))
# Directory to save the model
model_dir = "../mlp_model_nodrop_{}_{}_{}_{}_{}".format(layer_conf, feeder.window_size,
                                                   feeder.subsample, learning_rate, rho)
model_fn = build_mlp_model_fn(dense_layers, learning_rate, rho)
# Build the model
mlp_classifier = tf.estimator.Estimator(model_fn=model_fn, model_dir=model_dir)

In [ ]:
# Train the model specifing training steps the input and mode
mlp_classifier.train(input_fn=feeder.window_input_fn(mode='training'), steps=300000)

In [ ]:
# Call the evaluation functionality specifing input, mode and steps
eval_results = mlp_classifier.evaluate(input_fn=feeder.window_input_fn(mode='testing'), steps=10000)
print('MLP {}, lr={}, rho={}: {}'.format(dense_layers, learning_rate, rho, eval_results))

In [ ]:
# Print some results
fp_rate = eval_results['false_pos']/(eval_results['true_neg']+eval_results['false_pos'])
print('fp rate in percent {}'.format(fp_rate*100))
fn_rate = eval_results['false_neg']/(eval_results['true_pos']+eval_results['false_neg'])
print('fn rate in percent {}'.format(fn_rate*100))
acc =  (eval_results['true_pos']+eval_results['true_neg'])/(eval_results['true_neg']+eval_results['false_pos']+eval_results['true_pos']+eval_results['false_neg'])
print('accuracy {}'.format(acc))
prec = eval_results['true_pos']/(eval_results['true_pos']+eval_results['false_pos'])
print('precision {}'.format(prec))

In [ ]:
# Save the model for further use
mlp_classifier.export_savedmodel(os.path.join('\\saved_model\\no_drop{}_{}_{}_{}_{}'.format(layer_conf,
                                                                       feeder.window_size,
                                                                       feeder.subsample,
                                                                       learning_rate,
                                                                       rho)),
                                 feeder.serving_input_fn, as_text=False)